# Implementation with the AdaBoostClassifier


In [41]:
import os
import numpy as np

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

np.set_printoptions(precision=4)
seg_length = 500

TensorFlow version: 2.15.0


In [42]:
X = None
y = None

db1_pos = ['chfdb']
db1_neg = ['nsrdb', 'fantasia']
db2_pos = ['chf2db']
db2_neg = ['nsr2db']

for db in db1_pos:
    for record in os.listdir(f'data/{db}/rr'):
        rr = np.loadtxt(f'data/{db}/rr/{record}')
        n_segments = rr.shape[0] // seg_length
        rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
        labels = np.c_[np.ones(n_segments), np.zeros(n_segments)]

        if X is None or y is None:
            X = rr
            y = labels
        else:
            X = np.r_[rr, X]
            y = np.r_[labels, y]

for db in db1_neg:
    for record in os.listdir(f'data/{db}/rr'):
        rr = np.loadtxt(f'data/{db}/rr/{record}')
        n_segments = rr.shape[0] // seg_length
        rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
        labels = np.c_[np.zeros(n_segments), np.ones(n_segments)]

        if X is None or y is None:
            X = rr
            y = labels
        else:
            X = np.r_[rr, X]
            y = np.r_[labels, y]

X.shape, y.shape

((7397, 500), (7397, 2))

In [43]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# normalize?
# X = (X - X.mean()) / X.std()
X = preprocessing.scale(X, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=410)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=410)

# scaler = preprocessing.StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_valid = scaler.transform(X_valid)
# X_test = scaler.transform(X_test)

X_train.shape, X_valid.shape, X_test.shape

((5991, 500), (666, 500), (740, 500))

In [44]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=410)
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=410)

# X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_test.shape

In [45]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
X_test_scaled = min_max_scaler.transform(X_test)

print(min_max_scaler.scale_)

[0.0316 0.0465 0.0421 0.0601 0.053  0.0352 0.0476 0.0439 0.0441 0.0418
 0.0476 0.0482 0.054  0.0511 0.0395 0.0429 0.0431 0.0635 0.0538 0.0562
 0.0507 0.0504 0.0772 0.0592 0.0659 0.0612 0.0525 0.0316 0.064  0.0606
 0.0521 0.0325 0.0545 0.0535 0.0578 0.0528 0.0489 0.0524 0.048  0.0487
 0.0476 0.0412 0.0447 0.0366 0.0493 0.0563 0.0477 0.0373 0.0382 0.0308
 0.0541 0.0418 0.0447 0.0486 0.0541 0.0581 0.0551 0.0586 0.0589 0.0477
 0.0329 0.0623 0.0363 0.054  0.0499 0.0582 0.0559 0.0648 0.0552 0.0259
 0.0588 0.0491 0.053  0.0591 0.0455 0.0462 0.0581 0.051  0.0473 0.0592
 0.0525 0.0323 0.0295 0.0532 0.0486 0.0489 0.0561 0.0485 0.0506 0.0595
 0.0313 0.0458 0.054  0.0687 0.0563 0.0547 0.0477 0.0517 0.0411 0.0519
 0.042  0.0628 0.0482 0.0333 0.049  0.0623 0.0645 0.0328 0.0482 0.0468
 0.0559 0.0538 0.0295 0.0548 0.0604 0.055  0.0618 0.0554 0.0452 0.0354
 0.049  0.0591 0.0551 0.0411 0.0432 0.053  0.0556 0.0648 0.0622 0.0579
 0.0595 0.0583 0.058  0.0557 0.0348 0.0504 0.0508 0.0527 0.0581 0.0491
 0.044

In [46]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=101, n_estimators=300)
# y_train_reshaped = y_train.reshape(-1) #reshaped to be a 1D array for the AdaBoostClassifier
print(y_train.shape)
# y_train_flattened = y_train.flatten()
# print(y_train_flattened.shape)

y_train_reshaped = y_train[:, 0] #extracts the entire first column



model.fit(X_train_scaled,y_train_reshaped)

# training accuracy
print('Accuracy for the train data', model.score(X_train_scaled,y_train_reshaped))

(5991, 2)
Accuracy for the train data 0.8327491236855283


In [47]:
unique_values = np.unique(y_train_reshaped)
print(unique_values)

[0. 1.]


In [48]:
from sklearn import metrics

print("Type of y_test:", type(y_test))
print("Shape of y_test:", y_test.shape)

y_test_flattened = y_test[:, 0]
print(y_test_flattened.shape) #turn into a 1D array

unique_values = np.unique(y_test_flattened)
print(unique_values)

y_pred = model.predict(X_test_scaled)


# print("Type of y_pred:", type(y_pred))
# print("Shape of y_pred:", y_pred.shape)
unique_values = np.unique(y_pred)
print(unique_values)

print("Accuracy:", metrics.accuracy_score(y_test_flattened, y_pred))



# print("*** Confusion Matrix ***")                                  
# print(metrics.confusion_matrix(y_test, y_pred))

Type of y_test: <class 'numpy.ndarray'>
Shape of y_test: (740, 2)
(740,)
[0. 1.]
[0. 1.]
Accuracy: 0.677027027027027


In [49]:
# X_train.max(), X_train.min()

In [50]:
# X_train.mean(), X_train.std()

In [51]:
# inputs = tf.keras.layers.Input(shape=(500, 1))


# def inception_lstm(inputs):
#     a = tf.keras.layers.Bidirectional(
#         tf.keras.layers.LSTM(5, return_sequences=True))(inputs)
#     a = tf.keras.layers.LSTM(5, return_sequences=True)(a)

#     b = tf.keras.layers.Conv1D(5, kernel_size=1, padding='same')(inputs)
#     b = tf.keras.layers.Conv1D(5, kernel_size=3, padding='same')(b)

#     c = tf.keras.layers.Conv1D(5, kernel_size=1, padding='same')(inputs)
#     c = tf.keras.layers.Conv1D(5, kernel_size=5, padding='same')(c)

#     d = tf.keras.layers.MaxPool1D(3)(inputs)
#     d = tf.keras.layers.Conv1D(5, kernel_size=1)(d)

#     concat = tf.keras.layers.Concatenate(1)([a, b, c, d])
#     return concat


# layer1 = inception_lstm(inputs)
# layer2 = inception_lstm(layer1)
# dropout = tf.keras.layers.Dropout(0.2)(layer2)
# flattened = tf.keras.layers.Flatten()(dropout)
# output = tf.keras.layers.Dense(2, activation='sigmoid')(flattened)

# model = tf.keras.Model(inputs, output)
# model.summary()

In [52]:
# loss_fn = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer='adam',
#               loss=loss_fn,
#               metrics=['accuracy'])

In [53]:
# np.unique(y_test.argmax(axis=1), return_counts=True)

In [54]:
# y_pred = model.predict(X_test)

In [55]:
# np.unique(y_pred.argmax(axis=1), return_counts=True)

In [56]:
# # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid),
#           batch_size=128, epochs=100)

In [57]:
# model.save('model4.keras')

# model 3, minmax sample scaling w/ no early stopping (100 epochs)
# model 4, zero mean feature scaling w/ no early stopping (100 epochs)

In [58]:
# from sklearn.metrics import recall_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import accuracy_score

# y_pred = model.predict(X_test).argmax(axis=1)

In [59]:
# print(recall_score(y_test.argmax(axis=1), y_pred))
# print(precision_score(y_test.argmax(axis=1), y_pred))
# print(accuracy_score(y_test.argmax(axis=1), y_pred))